In [1]:
from autotst.reaction import *

In [2]:
log_file = "CC=C(C)C+[O]O_[CH2]C=C(C)C+OO_overall.log"
rxn_label = log_file.strip("_overall.log")

In [79]:
rxn = AutoTST_Reaction("CC=C(C)C+[O]O_[CH2]C=C(C)C+OO", "H_Abstraction")

final_log_file = rxn.label + "_overall.log"

from ase.io.gaussian import *
optimized_ase_ts = read_gaussian_out(log_file)

rxn.ts.ase_ts = optimized_ase_ts
rxn.ts.update_from_ase_ts()

from cclib.io import ccread
log_file_info = ccread(final_log_file)

rxn.vibrations = zip(log_file_info.vibfreqs, log_file_info.vibdisps)






rmg.py:72 __init__ WARNING Should only make one instance of RMGDatabase because it's stored as a module-level variable!
rmg.py:73 __init__ WARNING Unexpected behaviour may result!
thermo.py:839 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:839 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/libraries...
thermo.py:856 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-Py/../RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-Py/../RMG-database/input/transport/groups...
database.py:167 loadFamilies INFO Loading the user-speci

In [91]:
from cclib.io import ccread
log_file_info = ccread(final_log_file)

rxn.vibrations = zip(log_file_info.vibfreqs, log_file_info.vibdisps)

got_imaginary_frequency = False
for vib, displacements in rxn.vibrations:
    if vib < 0: # Finding the imaginary frequency
        got_imaginary_frequency = True
        post_vibration_ts = rxn.ts.ase_ts.copy() 
        post_vibration_ts.arrays["positions"] -= displacements
        
assert got_imaginary_frequency == True, "Did not find an imaginary frequency"
def percent_change(original,new):
    percent_change = (abs(new - original) / original) * 100
    return percent_change

results = []

for torsion in rxn.ts.torsions:
    i,j,k,l = torsion.indices
    before = rxn.ts.ase_ts.get_dihedral(i,j,k,l)
    after = copy_ts.get_dihedral(i,j,k,l)
    results.append(["Tor", torsion.indices, torsion.reaction_center, percent_change(before, after)])
    
for angle in rxn.ts.angles:
    i,j,k = angle.indices
    before = rxn.ts.ase_ts.get_angle(i,j,k)
    after = copy_ts.get_angle(i,j,k)
    results.append(["Ang", angle.indices, angle.reaction_center, percent_change(before, after)])

for bond in rxn.ts.bonds:
    i,j = bond.indices
    before = rxn.ts.ase_ts.get_distance(i,j)
    after = copy_ts.get_distance(i,j)
    results.append(["Bond", bond.indices, bond.reaction_center, percent_change(before, after)])

results = pd.DataFrame(results)
results.columns = ["type", "index", "center", "percent_change"]

if (np.log10(((results[results.center == "Yes"].mean()))) > np.log10(((results[results.center != "Yes"].mean()))) + 1).all():
    print "Vibrational analysis was successful"
    print True

else:
    print "Cannot reasonably say that we have arrived at a TS through vibrational analysis.\nRunning an IRC calc."
    print False

        

Vibrational analysis was successful
True


,type,index,center,percent_change
0,Tor,"(9, 0)",Yes,5.733631
1,Tor,"(9, 0)",Yes,69.775841
2,Tor,"(9, 0)",Yes,1798.334260
3,Ang,"(9, 0)",Yes,0.098216
4,Ang,"(9, 0)",Yes,36.648640
5,Ang,"(9, 0)",Yes,17.470714
6,Ang,"(9, 0)",Yes,1.148041
7,Ang,"(9, 0)",Yes,0.678533
8,Ang,"(9, 0)",Yes,7.093021
9,Ang,"(9, 0)",Yes,0.348713


In [5]:
from ase.io.gaussian import *
new_ase_ts = read_gaussian_out(log_file)

In [7]:
rxn.ts.ase_ts = new_ase_ts
rxn.ts.update_from_ase_ts()


In [8]:
rxn.ts.view_ts()

In [9]:
import cclib
from cclib.io import ccread
from ase.visualize import view

In [11]:
from cclib.io import ccread
log_file_info = ccread(log_file)
log_file_info

In [12]:
imaginary_displacements = log_file_info.vibdisps[0]
imaginary_displacements

array([[-0.04, -0.02,  0.02],
       [ 0.01,  0.  ,  0.01],
       [ 0.02,  0.  ,  0.  ],
       [-0.01,  0.01,  0.  ],
       [-0.04, -0.01,  0.03],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.01, -0.01],
       [ 0.01,  0.  ,  0.  ],
       [ 0.69,  0.25, -0.66],
       [ 0.06,  0.  , -0.07],
       [ 0.05,  0.03, -0.07],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ]])

In [13]:
copy_ts = rxn.ts.ase_ts.copy()
copy_ts

Atoms(symbols='O2C5H11', pbc=False)

In [14]:
copy_ts is rxn.ts.ase_ts

False

In [15]:
copy_ts.arrays["positions"]

array([[-3.11360028,  0.13591105, -0.7351732 ],
       [-3.94854798,  1.18903971, -0.47582964],
       [ 0.22522963,  0.40371981, -1.08033273],
       [ 0.92004854, -0.1404984 , -0.04722816],
       [-0.88609619,  1.34228768, -0.99666478],
       [ 0.66164011,  0.16229876,  1.4064651 ],
       [ 2.03476407, -1.12280726, -0.30380401],
       [-4.34657057,  1.37236634, -1.31879316],
       [ 0.48355681,  0.0769353 , -2.07435052],
       [-2.00144882,  0.66340832, -0.89022445],
       [-1.06903097,  1.90927311, -1.89854512],
       [-0.95623011,  1.95633538, -0.11251753],
       [ 1.54543065,  0.5965454 ,  1.86872944],
       [-0.16689127,  0.83889057,  1.56193728],
       [ 0.44150223, -0.75536243,  1.94632233],
       [ 2.18484254, -1.29875311, -1.36246503],
       [ 2.97210984, -0.76289313,  0.11470835],
       [ 1.82419429, -2.07866081,  0.17071886]])

In [16]:
copy_ts.arrays["positions"] -= imaginary_displacements
copy_ts.arrays["positions"]

array([[-3.07360028,  0.15591105, -0.7551732 ],
       [-3.95854798,  1.18903971, -0.48582964],
       [ 0.20522963,  0.40371981, -1.08033273],
       [ 0.93004854, -0.1504984 , -0.04722816],
       [-0.84609619,  1.35228768, -1.02666478],
       [ 0.66164011,  0.16229876,  1.4064651 ],
       [ 2.03476407, -1.12280726, -0.30380401],
       [-4.34657057,  1.36236634, -1.30879316],
       [ 0.47355681,  0.0769353 , -2.07435052],
       [-2.69144882,  0.41340832, -0.23022445],
       [-1.12903097,  1.90927311, -1.82854512],
       [-1.00623011,  1.92633538, -0.04251753],
       [ 1.54543065,  0.5965454 ,  1.86872944],
       [-0.16689127,  0.83889057,  1.56193728],
       [ 0.44150223, -0.75536243,  1.94632233],
       [ 2.18484254, -1.29875311, -1.36246503],
       [ 2.97210984, -0.76289313,  0.11470835],
       [ 1.82419429, -2.07866081,  0.17071886]])

In [33]:
view(rxn.ts.ase_ts)

In [32]:
view(copy_ts)

In [18]:
def percent_change(original,new):
    percent_change = (abs(new - original) / original) * 100
    return percent_change

In [39]:
results = []

In [40]:
for combo in rxn.ts.torsions:
    i,j,k,l = combo.indices
    print combo.reaction_center
    before = rxn.ts.ase_ts.get_dihedral(i,j,k,l)
    after = copy_ts.get_dihedral(i,j,k,l)
    #print before, after
    
    results.append(["Tor", combo.indices, combo.reaction_center, percent_change(before, after)])
    print percent_change(before, after)
    print

Close
5.733631148088095

Yes
69.7758405343532

Yes
1798.3342603225406



In [41]:
for combo in rxn.ts.angles:
    i,j,k = combo.indices
    print combo.reaction_center
    
    before = rxn.ts.ase_ts.get_angle(i,j,k)
    after = copy_ts.get_angle(i,j,k)
    
    #print int(before), int(after)

    results.append(["Ang", combo.indices, combo.reaction_center, percent_change(before, after)])
    print percent_change(before, after)
    print
    


No
0.0982157546914984

Yes
36.64864049951743

Close
17.47071411749732

No
1.1480412120612424

No
0.6785332829802169

Close
1.5228234143438597

No
7.093021094818691

No
0.3487126908331968

No
1.5214662993792993

No
1.081121474025241

No
0.19987941578763577

No
0.28275696754493435

No
0.45366611911775123

No
0.07081636313271636

No
0.08121301629569871

No
0.013226507995018035

No
0.5171209466871601

No
0.527702867237757

Close
5.132959141990106

Close
22.19498685534875

No
0.640154408944694

No
0.0

No
0.0

No
0.0

No
0.0

No
0.0

No
0.0



In [42]:
for combo in rxn.ts.bonds:
    i,j = combo.indices
    print combo.reaction_center
    
    before = rxn.ts.ase_ts.get_distance(i,j)
    after = copy_ts.get_distance(i,j)
    #print before, after
    results.append(["Bond", combo.indices, combo.reaction_center, percent_change(before, after)])
    
    print percent_change(before, after)
    print

Close
1.313948719720503

No
2.510156452535579

No
1.4404967738544294

Close
2.7463861408020964

No
0.22643861531843829

No
0.2511304275495249

No
0.9222912569555807

Yes
69.33491627821986

Close
5.95610774820469

Close
6.654547657677234

No
0.0

No
0.0

No
0.0

No
0.0

No
0.0

No
0.0

Yes
43.69740481186018



In [44]:
results = pd.DataFrame(results)
results.columns = ["type", "index", "center", "percent_change"]
results.sort_values("percent_change", ascending=False)

,type,index,center,percent_change
2,Tor,"(4, 9, 0, 1)",Yes,1798.334260
1,Tor,"(2, 4, 9, 0)",Yes,69.775841
37,Bond,"(4, 9)",Yes,69.334916
46,Bond,"(9, 0)",Yes,43.697405
4,Ang,"(0, 9, 4)",Yes,36.648640
22,Ang,"(9, 4, 11)",Close,22.194987
5,Ang,"(1, 0, 9)",Close,17.470714
9,Ang,"(2, 4, 10)",No,7.093021
39,Bond,"(4, 11)",Close,6.654548
38,Bond,"(4, 10)",Close,5.956108


In [53]:
b = results[results.center == "Yes"].mean()
print results[results.center == "No"].mean()
print results[results.center == "Close"].mean()

percent_change    0.609278
dtype: float64
percent_change    7.636234
dtype: float64


In [58]:
if ((float(results[results.center == "Yes"].mean()) > float(results[results.center == "Close"].mean())) and
    float(results[results.center == "Close"].mean()) > float(results[results.center == "No"].mean())):
    print "Yay"

Yay


In [74]:
if (np.log10(((results[results.center == "Yes"].mean()))) > np.log10(((results[results.center != "Yes"].mean()))) + 1).all():
    
    

Yay


In [75]:
np.log10(((results[results.center != "Yes"].mean()))) + 1

percent_change    1.325321
dtype: float64

In [64]:
np.log10((float(results[results.center == "No"].mean()))) + 1

0.784815236437582